### training models on species observation images

In [2]:
csv_path = 'C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images200_seed1117_attempt2_with_loc.csv'


temp = 'C:/Users/vjosv/master/top_100_images_with_location_data.csv'
import time
import torch 
import torchvision
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import os
import torch
import pandas as pd
import numpy as np
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.backends.cudnn as cudnn
import torch.nn as nn

import torch.nn.functional as F
import torch.nn.parallel
import torch.optim as optim
import torchvision
import torchvision.models as models
import sys

import ast
import shutil
from PIL import Image
from averagemeter import *
from models import *
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.autograd import Variable
from torch.utils.data import sampler
from torchvision import datasets
from torchvision import transforms
import torchvision
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


INPUT_SIZE = 224
NUM_CLASSES = 150
NUM_EPOCHS = 30
LEARNING_RATE = 1e-1
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
best_prec1 = 0
classes = []


train_path ='dataset/image_data/csv/combined_location_top_100_images_csv_train.csv'
test_path='dataset/image_data/csv/combined_location_top_100_images_csv_test.csv'
validation_path = 'dataset/image_data/csv/combined_location_top_100_images_csv_validation.csv'


True


In [2]:
transforms = [T.ToTensor(),T.Resize((INPUT_SIZE,INPUT_SIZE))]

composed = T.Compose(transforms)


class ImagesWithLocationDataset(Dataset):
    """Plant images with location data."""
    
    
    def __init__(self, csv_file, root_dir, transform=None):
    

        self.location_dataframe = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.location_dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.location_dataframe['path'].iloc[idx]
        
        image = io.imread(img_name)
        # location_data = self.from_np_array(self.location_dataframe['count_in_1000'].iloc[idx])
        target = np.array(self.location_dataframe['target'].iloc[idx])
        # landmarks = self.landmarks_frame.iloc[idx, 1:]
        # landmarks = np.array([landmarks], dtype=float).reshape(-1, 2)
        sample = {'image': image, 'location_data': None, 'target':target}
        sample = {'image': image, 'target':target}
        if self.transform:
            sample['image'] = self.transform(sample['image'])
            sample['target'] = torch.from_numpy(sample['target'])
            # sample['location_data'] = torch.from_numpy(location_data).float()
        return sample
    
    def from_np_array(self,array_string):
        array_string = ','.join(array_string.replace('[ ', '[').split())
        return np.array(ast.literal_eval(array_string))
    
    def name(self):
        return 'ImagesWithLocationDataset'


In [3]:

import ctypes
import platform

ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001


def _set_thread_execution(state):
    ctypes.windll.kernel32.SetThreadExecutionState(state)


def prevent_standby():
    if platform.system() == 'Windows':
        _set_thread_execution(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)


def allow_standby():
    if platform.system() == 'Windows':
        _set_thread_execution(ES_CONTINUOUS)


def long_running(func):
    def inner(*args, **kwargs):
        prevent_standby()
        result = func(*args, **kwargs)
        allow_standby()
        return result
    return inner

In [4]:
import copy

def train(train_loader, model, criterion, optimizer, epoch):

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, input in enumerate(train_loader):
        # measure data loading time
        # print(input['image'].shape)
        if USE_CUDA:
            image = input['image'].cuda(non_blocking=True)
            target = input['target'].cuda(non_blocking=True)
            # location_data = input['location_data'].cuda(non_blocking=True)
            # map_square = input['map_square'].cuda(non_blocking=True)
        else:
            image = input['image']
            target = input['target']
            # location_data = input['location_data']
            # map_square = input['map_square']
            
        input_var = torch.autograd.Variable(image)
        target_var = torch.autograd.Variable(target)
        # map_square_var = torch.autograd.Variable(map_square)
        # location_data_var = torch.autograd.Variable(location_data)
        # compute output
        # output = model(input_var,map_square)
        output = model(input_var)
        data_time.update(time.time() - end)

        
        # compute output
        # print(input_var.shape)
        # print(output)

        loss = criterion(output, target_var)
        # print(loss)
        # if i >10:
        #     break
        
        save_output_target_train(model,output,target_var,epoch,i)
        # measure accuracy and record loss
        prec1, prec5,conf = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data.item(), image.size(0))
        top1.update(prec1.item(), image.size(0))
        top5.update(prec5.item(), image.size(0))
        
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  '\Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1, top5=top5))
            
def validate(val_loader, model, criterion,epoch,save_output=False):
    # global output_aug_kde001, sample_pos, within_square, weight_tensor_kde001,zero_tensor_kde001,tt
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()


    conf = AverageMeter()
    class_correct = list(0. for i in range(200))
    class_total = list(0. for i in range(200))

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, input in enumerate(val_loader):
        

        if USE_CUDA:
            image = input['image'].cuda(non_blocking=True)
            target = input['target'].cuda(non_blocking=True)
            # location_data = input['location_data'].cuda(non_blocking=True)
            # map_square = input['map_square'].cuda(non_blocking=True)
        else:
            image = input['image']
            target = input['target']
            # location_data = input['location_data']
            # map_square = input['map_square']
        input_var = torch.autograd.Variable(image)
        target_var = torch.autograd.Variable(target)
        # location_data_var = torch.autograd.Variable(location_data)
        # map_square_var = torch.autograd.Variable(map_square)
        # compute output
        # output = model(input_var,map_square)
        output = model(input_var)
        loss = criterion(output, target)
        
        save_output_target_validate(model,output,target,epoch,i)
        

        # measure accuracy and record loss
        
        prec1, prec5, conf1 = accuracy(output.data, target, topk=(1, 5))
        

        losses.update(loss.data.item(), image.size(0))
        
        top1.update(prec1.item(), image.size(0))
        top5.update(prec5.item(), image.size(0))


        conf.update(conf1,1)
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:

            print('Test: [{0}/{1}]\n'.format(i, len(val_loader)))
            print('Time {batch_time.val:.3f} ({batch_time.avg:.3f})\n'.format(batch_time=batch_time))
            print('Loss {loss.val:.4f} ({loss.avg:.4f})\n'.format(loss=losses))
            print('Prec@1 {top1.val:.3f} ({top1.avg:.3f})\n'.format(top1=top1))
            print('Prec@5 {top5.val:.3f} ({top5.avg:.3f})\n'.format(top5=top5))

        # print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
        #       .format(top1=top1, top5=top5))
        


    return top1.avg

def adjust_learning_rate(optimizer, epoch, adjust_now, LEARNING_RATE=LEARNING_RATE):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LEARNING_RATE
    if (epoch-1) < 0:
        former_lr=optimizer.param_groups[0]['lr']

    former_lr = copy.copy(optimizer.param_groups[0]['lr'])
    if adjust_now:
        former_lr = copy.copy(optimizer.param_groups[0]['lr'])
        LEARNING_RATE = LEARNING_RATE*0.1
        lr = LEARNING_RATE
    print('\n[Learning Rate] {:0.6f}'.format(lr))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr != former_lr, LEARNING_RATE


def save_checkpoint(state, is_best, model):
    filename=f'saved_models/{model.name}_checkpoint.pth.tar'
    torch.save(state, filename)
    if is_best:
        print('\n[INFO] Saved Model to model_best.pth.tar')
        shutil.copyfile(filename, f'saved_models/{model.name}_model_best.pth.tar')
        
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred_t = pred.t()

    correct = pred_t.eq(target.view(1, -1).expand_as(pred_t))

    nb_classes = output.shape[1]

    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    _, preds = torch.max(output, 1)
    
    for t, p in zip(target.view(-1), preds.view(-1)):
        confusion_matrix[t.long(), p.long()] += 1

    # print(confusion_matrix)
    
    
    # print(confusion_matrix.diag()/confusion_matrix.sum(1))
        
    res = []
    for k in topk:
        correct_k = torch.sum(torch.reshape(correct[:k],(-1,) ).float()) #.double().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    res.append(confusion_matrix)
    return res

In [5]:
start_epoch =0
@long_running
def train_model(model,LEARNING_RATE = LEARNING_RATE,NUM_EPOCHS = NUM_EPOCHS,init_learning_rate = None,load_best=False ):
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
    train_loader = torch.utils.data.DataLoader(data_train, batch_size=16, shuffle=True, num_workers=0)
    val_loader = torch.utils.data.DataLoader(data_val, batch_size=16, shuffle=False, num_workers=0)
    
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                          momentum=0.9, weight_decay=1e-4, nesterov=True)
    criterion = nn.CrossEntropyLoss()

    args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
    if args_resume:
        if os.path.isfile(args_resume):
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))
            start_epoch = checkpoint['epoch']
        else:
            print("=> no checkpoint found at '{}'".format(args_resume))

            start_epoch = 0
            best_prec1 = 0
    print('\n[INFO] Training Started')
    five_epochs_since_best = False
    epochs_since_best = 0
    first_run = True 
    for epoch in range(start_epoch, NUM_EPOCHS ):
        
        if epochs_since_best >4:
            five_epochs_since_best = True
        
        if first_run and init_learning_rate:
            LEARNING_RATE = init_learning_rate
            print(LEARNING_RATE)
            adjusted_rate= False
        else:
            adjusted_rate, LEARNING_RATE = adjust_learning_rate(optimizer, epoch, five_epochs_since_best,LEARNING_RATE=LEARNING_RATE)
            five_epochs_since_best = False
        if adjusted_rate or (load_best and first_run):
            args_resume = f'saved_models/{model.name}_model_best.pth.tar'
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion,epoch,save_output=True)

        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best, model)
        print('\n[INFO] Saved Model to leafsnap_model.pth')
        torch.save(model, f'{model.name}_checkpoint.pth.tar')
        if is_best:
            epochs_since_best = 0
        else:
            epochs_since_best+=1
            
        first_run = False

def save_output_target_validate(model,output,target,epoch,batch):
        torch.save(output,f'saved_output/{model.name}_validating_output/output_e{epoch}_b{batch}')
        torch.save(target,f'saved_output/{model.name}_validating_output/target_b{batch}')
        
def save_output_target_train(model,output,target,epoch,batch):
        torch.save(output,f'saved_output/{model.name}_training_output/output_e{epoch}_b{batch}')

        torch.save(target,f'saved_output/{model.name}_training_output/target_e_{epoch}_b{batch}')
      

In [6]:

model = torchvision.models.vit_b_16(weights = 'ViT_B_16_Weights.IMAGENET1K_V1')

seed =1711
split_train =80
split_val =100-split_train
numb_targets = 100

torch.cuda.empty_cache() 


optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
criterion = nn.CrossEntropyLoss()

# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

model.heads[0] =nn.Linear(768 , 100, bias = True)


model.name = f'vit_b_16_v7_{numb_targets}targets_weights_seed{seed}_split{split_train}{split_val}_attempt2'


if USE_CUDA:
    model.cuda()

train_image_dataset = ImagesWithLocationDataset(train_path,'s',transform=composed)
test_image_dataset = ImagesWithLocationDataset(test_path,'s',transform=composed)
seed= 1212
torch.manual_seed(seed)


train_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=8, shuffle=False, num_workers=0)


USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    model.cuda()
if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
if not os.path.isdir(f'saved_output/{model.name}_training_output'):
    os.mkdir(f'saved_output/{model.name}_training_output')

    
@long_running
def train_model(model,LEARNING_RATE = LEARNING_RATE,NUM_EPOCHS = NUM_EPOCHS):

    train_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=12, shuffle=True, num_workers=0)
    val_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=12, shuffle=False, num_workers=0)
    
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                          momentum=0.9, weight_decay=1e-4, nesterov=True)
    criterion = nn.CrossEntropyLoss()

    args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
    if args_resume:
        if os.path.isfile(args_resume):
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))
            start_epoch = checkpoint['epoch']
        else:
            print("=> no checkpoint found at '{}'".format(args_resume))

            start_epoch = 0
            best_prec1 = 0
    print('\n[INFO] Training Started')
    
    for param_group in optimizer.param_groups:
        print(param_group['lr'])
    five_epochs_since_best = False
    epochs_since_best = 0
    for epoch in range(start_epoch, NUM_EPOCHS ):
        
        if epochs_since_best >4:
            five_epochs_since_best = True
            
        adjusted_rate,LEARNING_RATE = adjust_learning_rate(optimizer, epoch, five_epochs_since_best,LEARNING_RATE)
        print()
        if adjusted_rate:
            args_resume = f'saved_models/{model.name}_model_best.pth.tar'
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion,epoch,save_output=True)
        # prec1 = top1.avg
        # prec5 = top5.avg
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best, model)
        print('\n[INFO] Saved Model to leafsnap_model.pth')
        # torch.save(model, f'{model.name}_checkpoint.pth.tar')
        if is_best:
            epochs_since_best = 0
        else:
            epochs_since_best+=1
            
# for param in model.parameters():
#     param.requires_grad = False

# for param in model.location_fc2.parameters():
#     param.requires_grad = True
# for param in model.location_fc.parameters():
#     param.requires_grad = True   
# for param in model.linear_add.parameters():
#     param.requires_grad = True
    
if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 10)
train_model(model,LEARNING_RATE = 1e-3,NUM_EPOCHS = 15)
train_model(model,LEARNING_RATE = 1e-4,NUM_EPOCHS = 17)
train_model(model,LEARNING_RATE = 1e-5,NUM_EPOCHS = 19)

# train_model(model,LEARNING_RATE = 1e-5,NUM_EPOCHS = 22)

=> no checkpoint found at 'saved_models/vit_b_16_v7_100targets_weights_seed1711_split8020_attempt2_checkpoint.pth.tar'

[INFO] Training Started
0.01

[Learning Rate] 0.010000



C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torch\nn\functional.py:5440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:235.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch: [0][0/1334]	\Time 5.594 (5.594)	Data 4.562 (4.562)	Loss 4.2932 (4.2932)	Prec@1 0.000 (0.000)	Prec@5 16.667 (16.667)
Epoch: [0][100/1334]	\Time 0.311 (0.382)	Data 0.260 (0.297)	Loss 4.6783 (4.7449)	Prec@1 0.000 (0.825)	Prec@5 0.000 (4.620)
Epoch: [0][200/1334]	\Time 0.341 (0.359)	Data 0.291 (0.278)	Loss 4.5183 (4.6967)	Prec@1 0.000 (0.871)	Prec@5 8.333 (4.768)
Epoch: [0][300/1334]	\Time 0.357 (0.358)	Data 0.283 (0.278)	Loss 4.4854 (4.6761)	Prec@1 8.333 (1.135)	Prec@5 16.667 (4.817)
Epoch: [0][400/1334]	\Time 0.332 (0.358)	Data 0.271 (0.279)	Loss 4.6430 (4.6579)	Prec@1 0.000 (1.205)	Prec@5 8.333 (5.320)
Epoch: [0][500/1334]	\Time 0.342 (0.359)	Data 0.272 (0.281)	Loss 4.4953 (4.6235)	Prec@1 0.000 (1.480)	Prec@5 8.333 (6.404)
Epoch: [0][600/1334]	\Time 0.380 (0.358)	Data 0.310 (0.281)	Loss 4.0866 (4.5667)	Prec@1 8.333 (1.775)	Prec@5 33.333 (7.903)
Epoch: [0][700/1334]	\Time 0.433 (0.358)	Data 0.363 (0.281)	Loss 4.0424 (4.5057)	Prec@1 8.333 (2.294)	Prec@5 16.667 (9.736)
Epoch: [0][80

In [7]:
'combined_location_image_df_top100.csv'


torch.cuda.empty_cache() 
seed =1711
split_train =80
split_val =100-split_train
numb_targets = 100


model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)


optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
criterion = nn.CrossEntropyLoss()


# model = torchvision.models.vgg16(weights = True)





model.fc = nn.Linear(in_features=2048, out_features=100, bias=True)

model.name = f'resnet101_v4_{numb_targets}targets_weights_seed{seed}_split{split_train}{split_val}_attempt2'


if USE_CUDA:
    model.cuda()



train_image_dataset = ImagesWithLocationDataset(train_path,'s',transform=composed)
test_image_dataset = ImagesWithLocationDataset(test_path,'s',transform=composed)

torch.manual_seed(seed)


train_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=8, shuffle=False, num_workers=0)


USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    model.cuda()
if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
if not os.path.isdir(f'saved_output/{model.name}_training_output'):
    os.mkdir(f'saved_output/{model.name}_training_output')


    
@long_running
def train_model(model,LEARNING_RATE = LEARNING_RATE,NUM_EPOCHS = NUM_EPOCHS):
 
    train_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=12, shuffle=True, num_workers=0)
    val_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=12, shuffle=False, num_workers=0)
 
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                          momentum=0.9, weight_decay=1e-4, nesterov=True)
    criterion = nn.CrossEntropyLoss()

    args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
    if args_resume:
        if os.path.isfile(args_resume):
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))
            start_epoch = checkpoint['epoch']
        else:
            print("=> no checkpoint found at '{}'".format(args_resume))

            start_epoch = 0
            best_prec1 = 0
    
    print('\n[INFO] Training Started')
    five_epochs_since_best = False
    epochs_since_best = 0
    for epoch in range(start_epoch, NUM_EPOCHS ):
        
        if epochs_since_best >4:
            five_epochs_since_best = True
        
        adjusted_rate,LEARNING_RATE = adjust_learning_rate(optimizer, epoch, five_epochs_since_best,LEARNING_RATE)
        
        if adjusted_rate:
            print('Adjusted Learning rate resume from best model')
            args_resume = f'saved_models/{model.name}_model_best.pth.tar'
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion,epoch,save_output=True)

        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best, model)
        print('\n[INFO] Saved Model to leafsnap_model.pth')
        # torch.save(model, f'{model.name}_checkpoint.pth.tar')
        if is_best:
            epochs_since_best = 0
        else:
            epochs_since_best+=1
            
# for param in model.parameters():
#     param.requires_grad = False

# for param in model.location_fc2.parameters():
#     param.requires_grad = True
# for param in model.location_fc.parameters():
#     param.requires_grad = True   
# for param in model.linear_add.parameters():
#     param.requires_grad = True
    
if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 10)
train_model(model,LEARNING_RATE = 1e-3,NUM_EPOCHS = 15)
train_model(model,LEARNING_RATE = 1e-4,NUM_EPOCHS = 17)

Using cache found in C:\Users\vjosv/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


=> loading checkpoint 'saved_models/resnet101_v4_100targets_weights_seed1711_split8020_attempt2_checkpoint.pth.tar'
=> loaded checkpoint 'saved_models/resnet101_v4_100targets_weights_seed1711_split8020_attempt2_checkpoint.pth.tar' (epoch 2)

[INFO] Training Started

[Learning Rate] 0.010000
Epoch: [2][0/1334]	\Time 1.115 (1.115)	Data 0.675 (0.675)	Loss 1.5380 (1.5380)	Prec@1 33.333 (33.333)	Prec@5 91.667 (91.667)
Epoch: [2][100/1334]	\Time 0.384 (0.634)	Data 0.347 (0.570)	Loss 1.3338 (1.8650)	Prec@1 58.333 (48.350)	Prec@5 83.333 (79.868)
Epoch: [2][200/1334]	\Time 0.236 (0.503)	Data 0.208 (0.453)	Loss 2.0061 (1.8707)	Prec@1 50.000 (48.798)	Prec@5 66.667 (80.141)
Epoch: [2][300/1334]	\Time 0.306 (0.476)	Data 0.274 (0.429)	Loss 1.6923 (1.8769)	Prec@1 50.000 (49.031)	Prec@5 83.333 (80.260)
Epoch: [2][400/1334]	\Time 0.332 (0.485)	Data 0.293 (0.438)	Loss 1.6119 (1.8840)	Prec@1 58.333 (48.649)	Prec@5 91.667 (79.967)
Epoch: [2][500/1334]	\Time 0.409 (0.455)	Data 0.378 (0.411)	Loss 1.9450 (1.

In [6]:
'combined_location_image_df_top100.csv'

torch.cuda.empty_cache() 
# model = torchvision.models.vit_b_16(weights = 'VGG16_Weights.IMAGENET1K_V1')

seed =1711
split_train =80
split_val =100-split_train
numb_targets = 100




# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

model = torchvision.models.vgg19(weights = 'VGG19_Weights.IMAGENET1K_V1')

model.classifier[-1] = nn.Linear(4096 , 100, bias = True)
model.name = f'vgg19_v4_{numb_targets}targets_weights_seed{seed}_split{split_train}{split_val}_attempt2'




optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
criterion = nn.CrossEntropyLoss()

if USE_CUDA:
    model.cuda()



train_image_dataset = ImagesWithLocationDataset(train_path,'s',transform=composed)
test_image_dataset = ImagesWithLocationDataset(test_path,'s',transform=composed)

torch.manual_seed(seed)


train_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=8, shuffle=False, num_workers=0)


USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    model.cuda()
if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
if not os.path.isdir(f'saved_output/{model.name}_training_output'):
    os.mkdir(f'saved_output/{model.name}_training_output')


    
@long_running
def train_model(model,LEARNING_RATE = LEARNING_RATE,NUM_EPOCHS = NUM_EPOCHS):
 
    train_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=12, shuffle=True, num_workers=0)
    val_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=12, shuffle=False, num_workers=0)
    
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                          momentum=0.9, weight_decay=1e-4, nesterov=True)
    criterion = nn.CrossEntropyLoss()

    args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
    if args_resume:
        if os.path.isfile(args_resume):
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))
            start_epoch = checkpoint['epoch']
        else:
            print("=> no checkpoint found at '{}'".format(args_resume))

            start_epoch = 0
            best_prec1 = 0
    print('\n[INFO] Training Started')
    five_epochs_since_best = False
    epochs_since_best = 0
    for epoch in range(start_epoch, NUM_EPOCHS ):
        
        if epochs_since_best >4:
            five_epochs_since_best = True
            
        adjusted_rate,LEARNING_RATE = adjust_learning_rate(optimizer, epoch, five_epochs_since_best,LEARNING_RATE)
        
        if adjusted_rate:
            print('Adjusted Learning rate resume from best model')
            args_resume = f'saved_models/{model.name}_model_best.pth.tar'
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion,epoch,save_output=True)

        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best, model)
        print('\n[INFO] Saved Model to leafsnap_model.pth')
        # torch.save(model, f'{model.name}_checkpoint.pth.tar')
        if is_best:
            epochs_since_best = 0
        else:
            epochs_since_best+=1
            

    
if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 10)
train_model(model,LEARNING_RATE = 1e-3,NUM_EPOCHS = 15)
train_model(model,LEARNING_RATE = 1e-4,NUM_EPOCHS = 17)
train_model(model,LEARNING_RATE = 1e-5,NUM_EPOCHS = 19)

=> no checkpoint found at 'saved_models/vgg19_v4_100targets_weights_seed1711_split8020_attempt2_checkpoint.pth.tar'

[INFO] Training Started

[Learning Rate] 0.010000
Epoch: [0][0/1334]	\Time 5.086 (5.086)	Data 4.428 (4.428)	Loss 4.7167 (4.7167)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [0][100/1334]	\Time 0.321 (0.347)	Data 0.271 (0.287)	Loss 4.6161 (4.6231)	Prec@1 0.000 (1.073)	Prec@5 16.667 (5.693)
Epoch: [0][200/1334]	\Time 0.375 (0.329)	Data 0.322 (0.272)	Loss 4.5833 (4.6158)	Prec@1 0.000 (1.202)	Prec@5 8.333 (5.680)
Epoch: [0][300/1334]	\Time 0.366 (0.327)	Data 0.307 (0.272)	Loss 4.5972 (4.6150)	Prec@1 8.333 (1.246)	Prec@5 16.667 (5.537)
Epoch: [0][400/1334]	\Time 0.280 (0.326)	Data 0.230 (0.271)	Loss 4.6235 (4.6135)	Prec@1 0.000 (1.060)	Prec@5 0.000 (5.237)
Epoch: [0][500/1334]	\Time 0.304 (0.327)	Data 0.255 (0.272)	Loss 4.6412 (4.6085)	Prec@1 0.000 (1.031)	Prec@5 0.000 (5.539)
Epoch: [0][600/1334]	\Time 0.345 (0.325)	Data 0.292 (0.271)	Loss 4.5959 (4.6032)	Prec@1 0.000 (

In [8]:

# model = torchvision.models.vit_b_16(weights = 'VGG16_Weights.IMAGENET1K_V1')
torch.cuda.empty_cache() 
seed =1711
split_train =80
split_val =100-split_train
numb_targets = 100


model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)


optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
criterion = nn.CrossEntropyLoss()


# model = torchvision.models.vgg16(weights = True)





model.fc = nn.Linear(in_features=2048, out_features=100, bias=True)

model.name = f'resnet50_v3_{numb_targets}targets_weights_seed{seed}_split{split_train}{split_val}_attempt2'


if USE_CUDA:
    model.cuda()




train_image_dataset = ImagesWithLocationDataset(train_path,'s',transform=composed)
test_image_dataset = ImagesWithLocationDataset(test_path,'s',transform=composed)



train_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=8, shuffle=False, num_workers=0)


USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    model.cuda()
if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
if not os.path.isdir(f'saved_output/{model.name}_training_output'):
    os.mkdir(f'saved_output/{model.name}_training_output')


    
@long_running
def train_model(model,LEARNING_RATE = LEARNING_RATE,NUM_EPOCHS = NUM_EPOCHS):
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
    train_loader = torch.utils.data.DataLoader(data_train, batch_size=12, shuffle=True, num_workers=0)
    val_loader = torch.utils.data.DataLoader(data_val, batch_size=12, shuffle=False, num_workers=0)
    
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                          momentum=0.9, weight_decay=1e-4, nesterov=True)
    criterion = nn.CrossEntropyLoss()

    args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
    if args_resume:
        if os.path.isfile(args_resume):
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))
            start_epoch = checkpoint['epoch']
        else:
            print("=> no checkpoint found at '{}'".format(args_resume))

            start_epoch = 0
            best_prec1 = 0
    print('\n[INFO] Training Started')
    five_epochs_since_best = False
    epochs_since_best = 0
    for epoch in range(start_epoch, NUM_EPOCHS ):
        
        if epochs_since_best >4:
            five_epochs_since_best = True
            
        adjusted_rate,LEARNING_RATE = adjust_learning_rate(optimizer, epoch, five_epochs_since_best,LEARNING_RATE)
        
        if adjusted_rate:
            args_resume = f'saved_models/{model.name}_model_best.pth.tar'
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion,epoch,save_output=True)

        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best, model)
        print('\n[INFO] Saved Model to leafsnap_model.pth')
        # torch.save(model, f'{model.name}_checkpoint.pth.tar')
        if is_best:
            epochs_since_best = 0
        else:
            epochs_since_best+=1
            

    
if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 10)
train_model(model,LEARNING_RATE = 1e-3,NUM_EPOCHS = 15)
train_model(model,LEARNING_RATE = 1e-4,NUM_EPOCHS = 17)

Using cache found in C:\Users\vjosv/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


=> no checkpoint found at 'saved_models/resnet50_v3_100targets_weights_seed1711_split8020_attempt2_checkpoint.pth.tar'

[INFO] Training Started

[Learning Rate] 0.010000
Epoch: [0][0/1334]	\Time 0.352 (0.352)	Data 0.271 (0.271)	Loss 4.6459 (4.6459)	Prec@1 0.000 (0.000)	Prec@5 8.333 (8.333)
Epoch: [0][100/1334]	\Time 0.312 (0.295)	Data 0.284 (0.266)	Loss 4.6430 (4.6786)	Prec@1 0.000 (2.475)	Prec@5 0.000 (9.983)
Epoch: [0][200/1334]	\Time 0.334 (0.294)	Data 0.304 (0.265)	Loss 3.9758 (4.4698)	Prec@1 8.333 (3.400)	Prec@5 25.000 (14.552)
Epoch: [0][300/1334]	\Time 0.318 (0.295)	Data 0.290 (0.266)	Loss 3.1925 (4.2892)	Prec@1 25.000 (5.482)	Prec@5 33.333 (19.435)
Epoch: [0][400/1334]	\Time 0.262 (0.296)	Data 0.231 (0.268)	Loss 3.9803 (4.1321)	Prec@1 16.667 (7.066)	Prec@5 25.000 (23.608)
Epoch: [0][500/1334]	\Time 0.280 (0.297)	Data 0.251 (0.269)	Loss 3.3985 (3.9903)	Prec@1 8.333 (8.683)	Prec@5 50.000 (27.329)
Epoch: [0][600/1334]	\Time 0.324 (0.297)	Data 0.295 (0.269)	Loss 4.0070 (3.8744)	Pre

In [7]:
# model = torchvision.models.vit_b_16(weights = 'VGG16_Weights.IMAGENET1K_V1')

torch.cuda.empty_cache() 

seed =1711
split_train =80
split_val =100-split_train
numb_targets = 100


model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)


optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
criterion = nn.CrossEntropyLoss()


# model = torchvision.models.vgg16(weights = True)


model.fc = nn.Linear(in_features=512, out_features=100, bias=True)

model.name = f'resnet18_v3_{numb_targets}targets_weights_seed{seed}_split{split_train}{split_val}_attempt2'


train_image_dataset = ImagesWithLocationDataset(train_path,'s',transform=composed)
test_image_dataset = ImagesWithLocationDataset(test_path,'s',transform=composed)



train_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=8, shuffle=False, num_workers=0)


USE_CUDA = torch.cuda.is_available()

if USE_CUDA:
    model.cuda()
    
if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
if not os.path.isdir(f'saved_output/{model.name}_training_output'):
    os.mkdir(f'saved_output/{model.name}_training_output')


    
@long_running
def train_model(model,LEARNING_RATE = LEARNING_RATE,NUM_EPOCHS = NUM_EPOCHS):
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
    train_loader = torch.utils.data.DataLoader(data_train, batch_size=12, shuffle=False, num_workers=0)
    val_loader = torch.utils.data.DataLoader(data_val, batch_size=12, shuffle=False, num_workers=0)
    
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                          momentum=0.9, weight_decay=1e-4, nesterov=True)
    criterion = nn.CrossEntropyLoss()

    args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
    if args_resume:
        if os.path.isfile(args_resume):
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))
            start_epoch = checkpoint['epoch']
        else:
            print("=> no checkpoint found at '{}'".format(args_resume))

            start_epoch = 0
            best_prec1 = 0
    print('\n[INFO] Training Started')
    five_epochs_since_best = False
    epochs_since_best = 0
    for epoch in range(start_epoch, NUM_EPOCHS ):
        
        if epochs_since_best >4:
            five_epochs_since_best = True
            
        adjusted_rate,LEARNING_RATE = adjust_learning_rate(optimizer, epoch, five_epochs_since_best,LEARNING_RATE)
        
        if adjusted_rate:
            args_resume = f'saved_models/{model.name}_model_best.pth.tar'
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion,epoch,save_output=True)

        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best, model)
        print('\n[INFO] Saved Model to leafsnap_model.pth')
        # torch.save(model, f'{model.name}_checkpoint.pth.tar')
        if is_best:
            epochs_since_best = 0
        else:
            epochs_since_best+=1

    
if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 10)
train_model(model,LEARNING_RATE = 1e-3,NUM_EPOCHS = 15)
train_model(model,LEARNING_RATE = 1e-4,NUM_EPOCHS = 17)

Using cache found in C:\Users\vjosv/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


=> no checkpoint found at 'saved_models/resnet18_v3_100targets_weights_seed1711_split8020_attempt2_checkpoint.pth.tar'

[INFO] Training Started

[Learning Rate] 0.010000
Epoch: [0][0/1334]	\Time 6.378 (6.378)	Data 5.574 (5.574)	Loss 4.7821 (4.7821)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [0][100/1334]	\Time 0.197 (0.347)	Data 0.166 (0.312)	Loss 4.4415 (4.7731)	Prec@1 16.667 (4.703)	Prec@5 25.000 (16.419)
Epoch: [0][200/1334]	\Time 0.221 (0.322)	Data 0.194 (0.287)	Loss 3.5840 (4.4738)	Prec@1 33.333 (6.592)	Prec@5 41.667 (22.678)
Epoch: [0][300/1334]	\Time 0.250 (0.304)	Data 0.239 (0.271)	Loss 4.0377 (4.2284)	Prec@1 8.333 (8.583)	Prec@5 25.000 (27.796)
Epoch: [0][400/1334]	\Time 0.341 (0.292)	Data 0.310 (0.262)	Loss 2.5727 (4.0462)	Prec@1 33.333 (10.702)	Prec@5 58.333 (32.107)
Epoch: [0][500/1334]	\Time 0.218 (0.288)	Data 0.198 (0.258)	Loss 3.5704 (3.9070)	Prec@1 16.667 (12.242)	Prec@5 50.000 (35.329)
Epoch: [0][600/1334]	\Time 0.292 (0.285)	Data 0.263 (0.256)	Loss 4.2605 (3.799

In [8]:
# model = torchvision.models.vit_b_16(weights = 'VGG16_Weights.IMAGENET1K_V1')

torch.cuda.empty_cache() 

seed =1711
split_train =80
split_val =100-split_train
numb_targets = 100


model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)


optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
criterion = nn.CrossEntropyLoss()


# model = torchvision.models.vgg16(weights = True)


model.fc = nn.Linear(in_features=512, out_features=100, bias=True)

model.name = f'resnet34_v3_{numb_targets}targets_weights_seed{seed}_split{split_train}{split_val}_attempt2'


train_image_dataset = ImagesWithLocationDataset(train_path,'s',transform=composed)
test_image_dataset = ImagesWithLocationDataset(test_path,'s',transform=composed)



train_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=8, shuffle=False, num_workers=0)


USE_CUDA = torch.cuda.is_available()

if USE_CUDA:
    model.cuda()
    
if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
if not os.path.isdir(f'saved_output/{model.name}_training_output'):
    os.mkdir(f'saved_output/{model.name}_training_output')


    
@long_running
def train_model(model,LEARNING_RATE = LEARNING_RATE,NUM_EPOCHS = NUM_EPOCHS):
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
    train_loader = torch.utils.data.DataLoader(data_train, batch_size=12, shuffle=False, num_workers=0)
    val_loader = torch.utils.data.DataLoader(data_val, batch_size=12, shuffle=False, num_workers=0)
    
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                          momentum=0.9, weight_decay=1e-4, nesterov=True)
    criterion = nn.CrossEntropyLoss()

    args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
    if args_resume:
        if os.path.isfile(args_resume):
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))
            start_epoch = checkpoint['epoch']
        else:
            print("=> no checkpoint found at '{}'".format(args_resume))

            start_epoch = 0
            best_prec1 = 0
    print('\n[INFO] Training Started')
    five_epochs_since_best = False
    epochs_since_best = 0
    for epoch in range(start_epoch, NUM_EPOCHS ):
        
        if epochs_since_best >4:
            five_epochs_since_best = True
            
        adjusted_rate,LEARNING_RATE = adjust_learning_rate(optimizer, epoch, five_epochs_since_best,LEARNING_RATE)
        
        if adjusted_rate:
            args_resume = f'saved_models/{model.name}_model_best.pth.tar'
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion,epoch,save_output=True)

        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best, model)
        print('\n[INFO] Saved Model to leafsnap_model.pth')
        # torch.save(model, f'{model.name}_checkpoint.pth.tar')
        if is_best:
            epochs_since_best = 0
        else:
            epochs_since_best+=1
            
# for param in model.parameters():
#     param.requires_grad = False

# for param in model.location_fc2.parameters():
#     param.requires_grad = True
# for param in model.location_fc.parameters():
#     param.requires_grad = True   
# for param in model.linear_add.parameters():
#     param.requires_grad = True
    
if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 10)
train_model(model,LEARNING_RATE = 1e-3,NUM_EPOCHS = 15)
train_model(model,LEARNING_RATE = 1e-4,NUM_EPOCHS = 17)

Using cache found in C:\Users\vjosv/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


=> no checkpoint found at 'saved_models/resnet34_v3_100targets_weights_seed1711_split8020_attempt2_checkpoint.pth.tar'

[INFO] Training Started

[Learning Rate] 0.010000
Epoch: [0][0/1334]	\Time 0.405 (0.405)	Data 0.248 (0.248)	Loss 4.8029 (4.8029)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [0][100/1334]	\Time 0.187 (0.273)	Data 0.160 (0.245)	Loss 4.8459 (4.7995)	Prec@1 0.000 (2.393)	Prec@5 0.000 (8.663)
Epoch: [0][200/1334]	\Time 0.179 (0.270)	Data 0.159 (0.243)	Loss 3.7237 (4.5778)	Prec@1 16.667 (3.524)	Prec@5 41.667 (13.060)
Epoch: [0][300/1334]	\Time 0.290 (0.270)	Data 0.260 (0.243)	Loss 4.1381 (4.4134)	Prec@1 0.000 (4.790)	Prec@5 41.667 (17.027)
Epoch: [0][400/1334]	\Time 0.337 (0.269)	Data 0.307 (0.242)	Loss 3.5281 (4.2871)	Prec@1 16.667 (5.777)	Prec@5 33.333 (20.116)
Epoch: [0][500/1334]	\Time 0.219 (0.268)	Data 0.185 (0.241)	Loss 4.2634 (4.1720)	Prec@1 8.333 (7.019)	Prec@5 25.000 (23.470)
Epoch: [0][600/1334]	\Time 0.292 (0.268)	Data 0.262 (0.241)	Loss 4.5319 (4.0778)	Pre

In [10]:

torch.cuda.empty_cache() 

seed =1711
split_train =80
split_val =100-split_train
numb_targets = 100


model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)


optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
criterion = nn.CrossEntropyLoss()


# model = torchvision.models.vgg16(weights = True)


model.fc = nn.Linear(in_features=2048, out_features=100, bias=True)

model.name = f'resnet152_v3_{numb_targets}targets_weights_seed{seed}_split{split_train}{split_val}_attempt2'



train_image_dataset = ImagesWithLocationDataset(train_path,'s',transform=composed)
test_image_dataset = ImagesWithLocationDataset(test_path,'s',transform=composed)



train_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=8, shuffle=False, num_workers=0)

USE_CUDA = torch.cuda.is_available()

if USE_CUDA:
    model.cuda()
    
if not os.path.isdir(f'saved_output/{model.name}_validating_output'):
    os.mkdir(f'saved_output/{model.name}_validating_output')
if not os.path.isdir(f'saved_output/{model.name}_training_output'):
    os.mkdir(f'saved_output/{model.name}_training_output')


    
@long_running
def train_model(model,LEARNING_RATE = LEARNING_RATE,NUM_EPOCHS = NUM_EPOCHS):
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
    train_loader = torch.utils.data.DataLoader(data_train, batch_size=12, shuffle=True, num_workers=0)
    val_loader = torch.utils.data.DataLoader(data_val, batch_size=12, shuffle=False, num_workers=0)
    
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                          momentum=0.9, weight_decay=1e-4, nesterov=True)
    criterion = nn.CrossEntropyLoss()

    args_resume = f'saved_models/{model.name}_checkpoint.pth.tar'
    if args_resume:
        if os.path.isfile(args_resume):
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))
            start_epoch = checkpoint['epoch']
        else:
            print("=> no checkpoint found at '{}'".format(args_resume))

            start_epoch = 0
            best_prec1 = 0
    print('\n[INFO] Training Started')
    five_epochs_since_best = False
    epochs_since_best = 0
    for epoch in range(start_epoch, NUM_EPOCHS ):
        
        if epochs_since_best >4:
            five_epochs_since_best = True
            
        adjusted_rate,LEARNING_RATE = adjust_learning_rate(optimizer, epoch, five_epochs_since_best,LEARNING_RATE)
        
        if adjusted_rate:
            args_resume = f'saved_models/{model.name}_model_best.pth.tar'
            print("=> loading checkpoint '{}'".format(args_resume))
            checkpoint = torch.load(args_resume)
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args_resume, checkpoint['epoch']))

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)
        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion,epoch,save_output=True)

        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer': optimizer.state_dict(),
        }, is_best, model)
        print('\n[INFO] Saved Model to leafsnap_model.pth')
        # torch.save(model, f'{model.name}_checkpoint.pth.tar')
        if is_best:
            epochs_since_best = 0
        else:
            epochs_since_best+=1
            
# for param in model.parameters():
#     param.requires_grad = False

# for param in model.location_fc2.parameters():
#     param.requires_grad = True
# for param in model.location_fc.parameters():
#     param.requires_grad = True   
# for param in model.linear_add.parameters():
#     param.requires_grad = True
    
if USE_CUDA:
    model.cuda()
train_model(model,LEARNING_RATE = 1e-2,NUM_EPOCHS = 10)
train_model(model,LEARNING_RATE = 1e-3,NUM_EPOCHS = 15)
train_model(model,LEARNING_RATE = 1e-4,NUM_EPOCHS = 17)

Using cache found in C:\Users\vjosv/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\vjosv\anaconda3\envs\cuda_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


=> no checkpoint found at 'saved_models/resnet152_v3_100targets_weights_seed1711_split8020_attempt2_checkpoint.pth.tar'

[INFO] Training Started

[Learning Rate] 0.010000
Epoch: [0][0/1334]	\Time 0.358 (0.358)	Data 0.277 (0.277)	Loss 4.7197 (4.7197)	Prec@1 8.333 (8.333)	Prec@5 8.333 (8.333)
Epoch: [0][100/1334]	\Time 0.440 (0.324)	Data 0.396 (0.277)	Loss 4.6716 (4.6716)	Prec@1 8.333 (4.538)	Prec@5 25.000 (15.759)
Epoch: [0][200/1334]	\Time 0.544 (0.334)	Data 0.503 (0.287)	Loss 3.9841 (4.3811)	Prec@1 8.333 (6.592)	Prec@5 25.000 (21.559)
Epoch: [0][300/1334]	\Time 0.339 (0.334)	Data 0.295 (0.287)	Loss 3.2374 (4.1539)	Prec@1 16.667 (8.527)	Prec@5 58.333 (27.132)
Epoch: [0][400/1334]	\Time 0.302 (0.348)	Data 0.255 (0.301)	Loss 3.8924 (4.0048)	Prec@1 16.667 (9.435)	Prec@5 41.667 (30.258)
Epoch: [0][500/1334]	\Time 0.320 (0.348)	Data 0.266 (0.301)	Loss 3.3025 (3.8536)	Prec@1 8.333 (11.411)	Prec@5 25.000 (34.065)
Epoch: [0][600/1334]	\Time 0.382 (0.345)	Data 0.335 (0.298)	Loss 3.9442 (3.7427)


#### geo on species observation models:

In [ ]:
transforms = [T.ToTensor(),T.Resize((INPUT_SIZE,INPUT_SIZE))]

composed = T.Compose(transforms)


class ImagesWithLocationDataset(Dataset):
    """Plant images with location data."""
    
    
    def __init__(self, csv_file, root_dir, transform=None):
    

        self.location_dataframe = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.location_dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.location_dataframe['path'].iloc[idx]
        
        id_ = self.location_dataframe['Id'].iloc[idx]
        image = io.imread(img_name)
        path = img_name
        

        target = np.array(self.location_dataframe['target'].iloc[idx])

        sample = {'image': image, 'target':target, 'path':path, 'Id': id_}
        if self.transform:
            sample['image'] = self.transform(sample['image'])
            sample['target'] = torch.from_numpy(sample['target'])

        return sample
    
    
    def name(self):
        return 'ImagesWithLocationDataset'


In [ ]:


train_path ='dataset/image_data/csv/combined_location_top_100_images_csv_train.csv'
test_path='dataset/image_data/csv/combined_location_top_100_images_csv_test.csv'
validation_path = 'dataset/image_data/csv/combined_location_top_100_images_csv_validation.csv'

test_dat = pd.read_csv(test_path)
test_dat.columns
train_dat =  pd.read_csv(train_path)
test_dat_path = test_path

validation_dat = pd.read_csv(validation_path)

In [ ]:
import ctypes
import platform

ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001


def _set_thread_execution(state):
    ctypes.windll.kernel32.SetThreadExecutionState(state)


def prevent_standby():
    if platform.system() == 'Windows':
        _set_thread_execution(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)


def allow_standby():
    if platform.system() == 'Windows':
        _set_thread_execution(ES_CONTINUOUS)


def long_running(func):
    def inner(*args, **kwargs):
        prevent_standby()
        result = func(*args, **kwargs)
        allow_standby()
        return result
    return inner

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred_t = pred.t()

    correct = pred_t.eq(target.view(1, -1).expand_as(pred_t))

    nb_classes = output.shape[1]

    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    _, preds = torch.max(output, 1)
    
    for t, p in zip(target.view(-1), preds.view(-1)):
        confusion_matrix[t.long(), p.long()] += 1

    # print(confusion_matrix)
    
    
    # print(confusion_matrix.diag()/confusion_matrix.sum(1))
        
    res = []
    for k in topk:
        correct_k = torch.sum(torch.reshape(correct[:k],(-1,) ).float()) #.double().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    res.append(confusion_matrix)
    return res



In [ ]:


image_dataset = ImagesWithLocationDataset(test_dat_path,'s',transform=composed)


val_loader = torch.utils.data.DataLoader(image_dataset, batch_size=1, shuffle=False, num_workers=0)




torch.cuda.empty_cache() 
seed =1711
split_train =80
split_val =100-split_train
numb_targets = 100


model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)


optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
criterion = nn.CrossEntropyLoss()


# model = torchvision.models.vgg16(weights = True)





model.fc = nn.Linear(in_features=2048, out_features=100, bias=True)

model.name = f'resnet101_v4_{numb_targets}targets_weights_seed{seed}_split{split_train}{split_val}_attempt2'


# if USE_CUDA:
#     model.cuda()

args_resume = f'saved_models/{model.name}_model_best.pth.tar'
if args_resume:
    if os.path.isfile(args_resume):
        print("=> loading checkpoint '{}'".format(args_resume))
        checkpoint = torch.load(args_resume)
        start_epoch = checkpoint['epoch']
        best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(args_resume, checkpoint['epoch']))
        start_epoch = checkpoint['epoch']
if USE_CUDA:
    model.cuda()

In [ ]:
combined_df = pd.concat([test_dat,train_dat,validation_dat]) 
combined_df.columns

combined_df['height'] = combined_df['height'].fillna(combined_df['height'].mean())
combined_df['Nordkoordinat'] = combined_df['Nordkoordinat'].fillna(round(combined_df['Nordkoordinat'].mean(),0))

combined_df = combined_df[~combined_df['height'].isnull()]

ids = combined_df['Id']

list_h  = combined_df['height'].to_list()
list_n  = combined_df['Nordkoordinat'].to_list()
h_category = []
heigt_c = [100,300,600,900,1900]

for i in list_h:
    for j,k in enumerate(heigt_c):
        if i <k:
            h_category.append(j)
            break
combined_df['height_category'] = h_category

n_category = []
north_c = [6449347 +1*(2318191/5),6449347 +2*(2318191/5),6449347 +3*(2318191/5),6449347 +4*(2318191/5),6449347+10 +5*(2318191/5)]

for i in list_n:
    for j,k in enumerate(north_c):
        a = True
        if i <k:
            n_category.append(j)
            a=False
            break
    if a:
        print(i)

combined_df['north_category'] = n_category


In [ ]:
test_dat.columns



ids = combined_df['Id']
landscape = pd.get_dummies(combined_df['landscape'],drop_first =True)
treeline  = pd.get_dummies(combined_df['treeline'],drop_first =True)
arskogbon = pd.get_dummies(combined_df['arskogbon'],drop_first =True)
artreslag = pd.get_dummies(combined_df['artreslag'],drop_first =True)
   

arjordbr = pd.get_dummies(combined_df['arjordbr'],drop_first =True)
arveget  = pd.get_dummies(combined_df['arveget'],drop_first =True)
Fylke = pd.get_dummies(combined_df['Fylke'],drop_first =True)
north_category = pd.get_dummies(combined_df['north_category'],drop_first =True)
height_category = pd.get_dummies(combined_df['height_category'],drop_first =True)
landscape_subtype = pd.get_dummies(combined_df['landscape_subtype'],drop_first =True)

df = pd.concat([ids,landscape,treeline,arskogbon,artreslag,arjordbr,arveget,Fylke,landscape_subtype,height_category,north_category],axis=1)

df.columns = df.columns.astype(str)


multinomial_logisticmodel = pickle.load(open('multinomial_logisticmodel1.sav', 'rb'))
# multinomial_logisticmodel = pickle.load(open('multinomial_logisticmodel1_validate.sav', 'rb'))

### Run this square for ViT model, skip if not

In [ ]:
#load ViT model
torch.cuda.empty_cache() 

model = torchvision.models.vit_b_16(weights = 'ViT_B_16_Weights.IMAGENET1K_V1')

seed =1711
split_train =80
split_val =100-split_train
numb_targets = 100

torch.cuda.empty_cache() 


optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE,
                      momentum=0.9, weight_decay=1e-4, nesterov=True)
criterion = nn.CrossEntropyLoss()

# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

model.heads[0] =nn.Linear(768 , 100, bias = True)
model.name = f'vit_b_16_v5_{numb_targets}targets_weights_seed{seed}_split{split_train}{split_val}_attempt2'
#new version
model.name = f'vit_b_16_v7_{numb_targets}targets_weights_seed{seed}_split{split_train}{split_val}_attempt2'



if USE_CUDA:
    model.cuda()

# args_resume = f'C:/Users/vjosv/master/Deep-Leafsnap/saved_models/{model.name}_model_best.pth.tar'
# print("=> loading checkpoint '{}'".format(args_resume))
# checkpoint = torch.load(args_resume)
# best_prec1 = checkpoint['best_prec1']
# model.load_state_dict(checkpoint['state_dict'])
# print("=> loaded checkpoint '{}' (epoch {})"
#       .format(args_resume, checkpoint['epoch']))
# epoch = checkpoint['epoch']




image_dataset = ImagesWithLocationDataset(test_dat_path,'s',transform=composed)




USE_CUDA = torch.cuda.is_available()

if USE_CUDA:
    model.cuda()

args_resume = f'saved_models/{model.name}_model_best.pth.tar'
if args_resume:
    if os.path.isfile(args_resume):
        print("=> loading checkpoint '{}'".format(args_resume))
        checkpoint = torch.load(args_resume)
        start_epoch = checkpoint['epoch']
        best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(args_resume, checkpoint['epoch']))
        start_epoch = checkpoint['epoch']
if USE_CUDA:
    model.cuda()
    

# image_dataset = ImagesWithLocationDataset(test_dat_path,'s',transform=composed)



image_dataset = ImagesWithLocationDataset(test_dat_path,'s',transform=composed)


val_loader = torch.utils.data.DataLoader(image_dataset, batch_size=1, shuffle=False, num_workers=0)



In [ ]:

image_dataset = ImagesWithLocationDataset(test_dat_path,'s',transform=composed)


val_loader = torch.utils.data.DataLoader(image_dataset, batch_size=1, shuffle=False, num_workers=0)




better_1_org = {}
better_5 = {}
better_10 = {}


print(model.name)

torch.cuda.empty_cache()
model.eval()

top1 = AverageMeter()
top5 = AverageMeter()
top10 = AverageMeter()

top1_aug = AverageMeter()
top5_aug = AverageMeter()
top10_aug = AverageMeter()



above_10_list = [0, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 23, 24, 25, 27, 29, 30, 31, 32, 35, 40, 41, 42, 43, 44, 46, 47, 50, 52, 53, 55, 57, 60, 61, 63, 64, 65, 68, 70, 71, 72, 73, 74, 75, 76, 78, 79, 81, 83, 86, 87]
for i,input in enumerate(tqdm(val_loader)):
    

    # print(i)

    if USE_CUDA:
        image = input['image'].cuda(non_blocking=True)
        target = input['target'].cuda(non_blocking=True)
        # location_data = input['location_data'].cuda(non_blocking=True)
        # map_square = input['map_square'].cuda(non_blocking=True)
    else:
        image = input['image']
        target = input['target']
        # location_data = input['location_data']
        # map_square = input['map_square']
    input_var = torch.autograd.Variable(image)
    target_var = torch.autograd.Variable(target)
    # location_data_var = torch.autograd.Variable(location_data)
    # map_square_var = torch.autograd.Variable(map_square)
    # compute output
    # output = model(input_var,map_square)
    output = model(input_var)
    
    output_aug = torch.nn.functional.softmax(output, dim=1)
    
    original_prediction = accuracy(output.data, target, topk=(1, 5,10))
    
    top1.update(original_prediction[0].item(), image.size(0))
    top5.update(original_prediction[1].item(), image.size(0))
    top10.update(original_prediction[2].item(), image.size(0))
    original_prediction = [int(op) for op in original_prediction[:-1]]
    original_prediction
    # for mul in [0.1,0.2,0.3,0.4,0.5]:
    
    for mul in [0.9,0.8,0.7]:#,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:# ['in_100', 'in_200', 'in_500', 'in_1000', 'in_1500']:
        output_aug = output #torch.nn.functional.softmax(output, dim=1)
        if mul not in better_1_org:
            better_1_org[mul] = [AverageMeter(),AverageMeter(),AverageMeter()]

        id_ = input['Id']

        # output_aug[0] = output_aug[0]+ (0.45/2)*torch.from_numpy(np.array(ast.literal_eval(test_dat[test_dat['path']==input['path'][0]]['in_1000'].values[0]))).cuda()
        
        
        a= torch.from_numpy(np.array(ast.literal_eval(test_dat[test_dat['path']==input['path'][0]]['in_200'].values[0]))).cuda()
        for i in range(len(a)):
            if a[i]==0:
                a[i]=mul
        output_aug[0] = output_aug[0]*a
        output_aug = nn.functional.normalize(output_aug)
        # mul_log = multinomial_logisticmodel.predict_proba(df[df['Id']==int(id_)].loc[:, df.columns != 'Id'])[0]
        
        # ones = np.zeros(100)
        # null_val = np.argsort(mul_log)[:mul]
        # null_val = [i for i in null_val if i not in above_10_list]
        # ones[null_val]=0

        # output_aug[0][null_val]=0
        # mul_log[null_val]=0
        # output_aug[0] = output_aug[0]

        



        augmented_prediction = accuracy(output_aug.data, target, topk=(1, 5, 10))
        top1_aug.update(augmented_prediction[0].item(), image.size(0))
        top5_aug.update(augmented_prediction[1].item(), image.size(0))
        top10_aug.update(augmented_prediction[2].item(), image.size(0))
        
        better_1_org[mul][0].update(augmented_prediction[0].item(), image.size(0))
        better_1_org[mul][1].update(augmented_prediction[1].item(), image.size(0))
        better_1_org[mul][2].update(augmented_prediction[2].item(), image.size(0))
        
        

In [ ]:
#printing result of augmented model
print(top1.avg)
print(top5.avg)
print(top10.avg)
print('--------')
for k in better_1_org:
    print(k)
    # for s in better_1_org[k]:
    print('&'+' 1','& ',round(better_1_org[k][0].avg,3),' & ',round(round(better_1_org[k][0].avg,3)-round(top1.avg,3),3),"\\\\")
    print('&'+' 5','& ',round(better_1_org[k][1].avg,3),' & ',round(round(better_1_org[k][1].avg,3)-round(top5.avg,3),3),"\\\\")
    print('&'+' 10','& ',round(better_1_org[k][2].avg,3),'&',round(round(better_1_org[k][2].avg,3)-round(top10.avg,3),3),"\\\\")
    print("\\midrule")


## Trying to weight with relative frequency in the area around the observation


### In Area with Floor multiplied

In [ ]:

better_1_1 = {}
better_5 = {}
better_10 = {}

# accuracy_per_target

image_dataset = ImagesWithLocationDataset(test_dat_path,'s',transform=composed)




import warnings
warnings.filterwarnings("ignore")



torch.cuda.empty_cache()
model.eval()

top1 = AverageMeter()
top5 = AverageMeter()
top10 = AverageMeter()

top1_aug = AverageMeter()
top5_aug = AverageMeter()
top10_aug = AverageMeter()

for i,input in enumerate(tqdm(val_loader)):
    

    # print(i)

    if USE_CUDA:
        image = input['image'].cuda(non_blocking=True)
        target = input['target'].cuda(non_blocking=True)
        # location_data = input['location_data'].cuda(non_blocking=True)
        # map_square = input['map_square'].cuda(non_blocking=True)
    else:
        image = input['image']
        target = input['target']
        # location_data = input['location_data']
        # map_square = input['map_square']
    input_var = torch.autograd.Variable(image)
    target_var = torch.autograd.Variable(target)

    output = model(input_var)
    
    output_aug = torch.nn.functional.softmax(output, dim=1)
    
    original_prediction = accuracy(output.data, target, topk=(1, 5,10))
    
    top1.update(original_prediction[0].item(), image.size(0))
    top5.update(original_prediction[1].item(), image.size(0))
    top10.update(original_prediction[2].item(), image.size(0))
    original_prediction = [int(op) for op in original_prediction[:-1]]
    original_prediction
    # for mul in [0.1,0.2,0.3,0.4,0.5]:
    for in_val in ['in_500','in_1000','in_1500']:
        if in_val not in better_1_1:
            better_1_1[in_val]={}
        for mul in [0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1,0]:#0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
            if mul not in better_1_1[in_val]:
                better_1_1[in_val][mul] = [AverageMeter(),AverageMeter(),AverageMeter()]

            id_ = input['Id']
            output_aug = torch.nn.functional.softmax(output, dim=1)

            # output_aug[0] = output_aug[0]+ 0.5*0.45*torch.from_numpy(np.array(ast.literal_eval(test_dat[test_dat['path']==input['path'][0]]['in_1000'].values[0]))).cuda()

            a= torch.from_numpy(np.array(ast.literal_eval(test_dat[test_dat['path']==input['path'][0]][in_val].values[0]))).cuda()
            for i in range(len(a)):
                if a[i]==0:
                    a[i]=mul
            output_aug[0] = output_aug[0]*a
            output_aug = nn.functional.normalize(output_aug)

            # mul_log = multinomial_logisticmodel.predict_proba(df[df['Id']==int(id_)].loc[:, df.columns != 'Id'])[0]

            ones = np.zeros(100)
            # null_val = np.argsort(mul_log)[60:]
            # null_val = [i for i in null_val if i in above_10_list]
            # ones[null_val]=1
            # output_aug[0] = output_aug[0]+ mul*torch.from_numpy(mul_log).cuda()

            # freq_square = test_dat[test_dat['Id']==int(input['Id'])][square_res].iloc[0]
            # freq_square = test_dat[test_dat['Id']==int(input['Id'])][square_res].iloc[0]


            # list_surrounding_squares = list(get_sourounding_squares(freq_square,square_res,test_dat)[square_res].unique())
            # try:

    #         freq_square_l = list(get_sourounding_squares(freq_square,square_res,test_dat)[square_res].unique())                
    #         freq_square_l_new = [f_s_l for f_s_l in freq_square_l if f_s_l in res_dat.columns]

    #         np_a = np.zeros(100)
    #         for c in res_dat[freq_square_l_new].columns:
    #             np_a += np.array(res_dat[freq_square_l_new][c])
            # np_freq_square = np.array(res_dat[freq_square])
    # 
            # output_aug[0] = output_aug[0]+ mul*torch.from_numpy(np_a).cuda()


            augmented_prediction = accuracy(output_aug.data, target, topk=(1, 5, 10))

            better_1_1[in_val][mul][0].update(augmented_prediction[0].item(), image.size(0))
            better_1_1[in_val][mul][1].update(augmented_prediction[1].item(), image.size(0))
            better_1_1[in_val][mul][2].update(augmented_prediction[2].item(), image.size(0))

warnings.filterwarnings("default")



In [ ]:
# test_dat[test_dat['Id']==int(input['Id'])][square_res][1]

print(top1.avg)
print(top5.avg)
print(top10.avg)
# print(better_1)
for s in better_1_1:
    print(s)
    for k in better_1_1[s]:
        print(k,' &'+' 1','& ',round(better_1_1[s][k][0].avg,3),' & ',round(round(better_1_1[s][k][0].avg,3)-round(top1.avg,3),3),"\\\\")
        print('&'+' 5','& ',round(better_1_1[s][k][1].avg,3),' & ',round(round(better_1_1[s][k][1].avg,3)-round(top5.avg,3),3),"\\\\")
        print('&'+' 10','& ',round(better_1_1[s][k][2].avg,3),'&',round(round(better_1_1[s][k][2].avg,3)-round(top10.avg,3),3),"\\\\")
        print("\\midrule")

## Added In Area 

In [ ]:
better_1_2 = {}
better_5 = {}
better_10 = {}

# accuracy_per_target

image_dataset = ImagesWithLocationDataset(test_dat_path,'s',transform=composed)

print(model.name)


import warnings
warnings.filterwarnings("ignore")



torch.cuda.empty_cache()
model.eval()

top1 = AverageMeter()
top5 = AverageMeter()
top10 = AverageMeter()

top1_aug = AverageMeter()
top5_aug = AverageMeter()
top10_aug = AverageMeter()

for i,input in enumerate(tqdm(val_loader)):
    

    # print(i)

    if USE_CUDA:
        image = input['image'].cuda(non_blocking=True)
        target = input['target'].cuda(non_blocking=True)
        # location_data = input['location_data'].cuda(non_blocking=True)
        # map_square = input['map_square'].cuda(non_blocking=True)
    else:
        image = input['image']
        target = input['target']
        # location_data = input['location_data']
        # map_square = input['map_square']
    input_var = torch.autograd.Variable(image)
    target_var = torch.autograd.Variable(target)
    # location_data_var = torch.autograd.Variable(location_data)
    # map_square_var = torch.autograd.Variable(map_square)
    # compute output
    # output = model(input_var,map_square)
    output = model(input_var)
    
    output_aug = torch.nn.functional.softmax(output, dim=1)
    
    original_prediction = accuracy(output.data, target, topk=(1, 5,10))
    
    top1.update(original_prediction[0].item(), image.size(0))
    top5.update(original_prediction[1].item(), image.size(0))
    top10.update(original_prediction[2].item(), image.size(0))
    original_prediction = [int(op) for op in original_prediction[:-1]]
    original_prediction
    # for mul in [0.1,0.2,0.3,0.4,0.5]:
    for in_val in ['in_200','in_500','in_1000','in_1500']:
        if in_val not in better_1_2:
            better_1_2[in_val]={}
        for mul in [0.6,0.5,0.45,0.4,0.3,0.2,0.1]:#0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
            if mul not in better_1_2[in_val]:
                better_1_2[in_val][mul] = [AverageMeter(),AverageMeter(),AverageMeter()]

            id_ = input['Id']
            output_aug = torch.nn.functional.softmax(output, dim=1)

            output_aug[0] = output_aug[0]+ mul*torch.from_numpy(np.array(ast.literal_eval(test_dat[test_dat['path']==input['path'][0]][in_val].values[0]))).cuda()

  
            ones = np.zeros(100)



            augmented_prediction = accuracy(output_aug.data, target, topk=(1, 5, 10))

            better_1_2[in_val][mul][0].update(augmented_prediction[0].item(), image.size(0))
            better_1_2[in_val][mul][1].update(augmented_prediction[1].item(), image.size(0))
            better_1_2[in_val][mul][2].update(augmented_prediction[2].item(), image.size(0))

warnings.filterwarnings("default")



In [ ]:
print(top1.avg)
print(top5.avg)
print(top10.avg)
# print(better_1)
for s in better_1_2:
    print(s)
    for k in better_1_2[s]:
        print(k,' &'+' 1','& ',round(better_1_2[s][k][0].avg,3),' & ',round(round(better_1_2[s][k][0].avg,3)-round(top1.avg,3),3),"\\\\")
        print('&'+' 5','& ',round(better_1_2[s][k][1].avg,3),' & ',round(round(better_1_2[s][k][1].avg,3)-round(top5.avg,3),3),"\\\\")
        print('&'+' 10','& ',round(better_1_2[s][k][2].avg,3),'&',round(round(better_1_2[s][k][2].avg,3)-round(top10.avg,3),3),"\\\\")
        print("\\midrule")

In [ ]:
better_1_3 = {}
better_5 = {}
better_10 = {}

# accuracy_per_target

image_dataset = ImagesWithLocationDataset(test_dat_path,'s',transform=composed)




import warnings
warnings.filterwarnings("ignore")

print(model.name)

torch.cuda.empty_cache()
model.eval()

top1 = AverageMeter()
top5 = AverageMeter()
top10 = AverageMeter()

top1_aug = AverageMeter()
top5_aug = AverageMeter()
top10_aug = AverageMeter()

for i,input in enumerate(tqdm(val_loader)):
    

    # print(i)

    if USE_CUDA:
        image = input['image'].cuda(non_blocking=True)
        target = input['target'].cuda(non_blocking=True)
        # location_data = input['location_data'].cuda(non_blocking=True)
        # map_square = input['map_square'].cuda(non_blocking=True)
    else:
        image = input['image']
        target = input['target']
        # location_data = input['location_data']
        # map_square = input['map_square']
    input_var = torch.autograd.Variable(image)
    target_var = torch.autograd.Variable(target)
    # location_data_var = torch.autograd.Variable(location_data)
    # map_square_var = torch.autograd.Variable(map_square)
    # compute output
    # output = model(input_var,map_square)
    output = model(input_var)
    
    output_aug = torch.nn.functional.softmax(output, dim=1)
    
    original_prediction = accuracy(output.data, target, topk=(1, 5,10))
    
    top1.update(original_prediction[0].item(), image.size(0))
    top5.update(original_prediction[1].item(), image.size(0))
    top10.update(original_prediction[2].item(), image.size(0))
    original_prediction = [int(op) for op in original_prediction[:-1]]
    original_prediction
    # for mul in [0.1,0.2,0.3,0.4,0.5]:
    for in_val in ['in_500']:
        if in_val not in better_1_3:
            better_1_3[in_val]={}
            

        for mul in [1.8,2]:# [0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.4,1.5,1.6]:
            if mul not in better_1_3[in_val]:
                better_1_3[in_val][mul] = [AverageMeter(),AverageMeter(),AverageMeter()]

            id_ = input['Id']
            output_aug = torch.nn.functional.softmax(output, dim=1)

            # output_aug[0] = output_aug[0]+ mul*torch.from_numpy(np.array(ast.literal_eval(test_dat[test_dat['path']==input['path'][0]][in_val].values[0]))).cuda()

            # output_aug[0] = output_aug[0]+ (0.45/2)*torch.from_numpy(np.array(ast.literal_eval(test_dat[test_dat['path']==input['path'][0]]['in_1000'].values[0]))).cuda()
            # output_aug = nn.functional.normalize(output_aug)
            
            ##top-40 logistic
            
            mul_log = multinomial_logisticmodel.predict_proba(df[df['Id']==int(id_)].loc[:, df.columns != 'Id'])[0]
            ones = np.zeros(100)
            null_val = np.argsort(mul_log)[60:]
            ones[null_val]=1
        
            output_aug[0] = output_aug[0]+ mul*torch.from_numpy(mul_log).cuda()
            

            augmented_prediction = accuracy(output_aug.data, target, topk=(1, 5, 10))

            better_1_3[in_val][mul][0].update(augmented_prediction[0].item(), image.size(0))
            better_1_3[in_val][mul][1].update(augmented_prediction[1].item(), image.size(0))
            better_1_3[in_val][mul][2].update(augmented_prediction[2].item(), image.size(0))

warnings.filterwarnings("default")


In [22]:
print(top1.avg)
print(top5.avg)
print(top10.avg)
# print(better_1)
for s in better_1_3:
    print(s)
    for k in better_1_3[s]:
        print(k,' &'+' 1','& ',round(better_1_3[s][k][0].avg,3),' & ',round(round(better_1_3[s][k][0].avg,3)-round(top1.avg,3),3),"\\\\")
        print('&'+' 5','& ',round(better_1_3[s][k][1].avg,3),' & ',round(round(better_1_3[s][k][1].avg,3)-round(top5.avg,3),3),"\\\\")
        print('&'+' 10','& ',round(better_1_3[s][k][2].avg,3),'&',round(round(better_1_3[s][k][2].avg,3)-round(top10.avg,3),3),"\\\\")
        print("\\midrule")

0.1

### Frequency weighted with floor

In [ ]:
## Frequency weighted with floor
##
##
better_1_4 = {}
better_5 = {}
better_10 = {}

import warnings
warnings.filterwarnings("ignore")

def get_sourounding_squares(square,resolution, dat):
    if isinstance(square, str):
        square= literal_eval(square)
    
    return_list = []
    for i in [-1,0,1]:
        for j in [-1,0,1]:
            return_list.append(str([square[0]+i,square[1]+j]))
    
    return dat[dat[resolution].isin(return_list)]

torch.cuda.empty_cache()
model.eval()


top1_aug = AverageMeter()
top5_aug = AverageMeter()
top10_aug = AverageMeter()

sq = ['square_20x20', 'square_15x15','square_10x10', 'square_30x30', 'square_40x40', 'square_50x50','square_60x60', 'square_70x70', 'square_80x80', 'square_90x90','square_100x100']
sq = ['square_150x150','square_200x200']


sq = ['squares_8','squares_20','squares_15', 'squares_10', 'squares_30', 'squares_40', 'squares_50','squares_60', 'squares_70', 'squares_80', 'squares_90', 'squares_100']
sq = ['squares_60','squares_70', 'squares_80', 'squares_90', 'squares_100']
sq =['squares_100']
above_10_list = [0, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 23, 24, 25, 27, 29, 30, 31, 32, 35, 40, 41, 42, 43, 44, 46, 47, 50, 52, 53, 55, 57, 60, 61, 63, 64, 65, 68, 70, 71, 72, 73, 74, 75, 76, 78, 79, 81, 83, 86, 87]
for square_res in sq:
    better_1_4[square_res] = {}
    res_dat = pd.read_csv(f'C:/Users/vjosv/master/Deep-Leafsnap/{square_res}_relative_freq_min.csv')
    top1 = AverageMeter()
    top5 = AverageMeter()
    top10 = AverageMeter()

    for i,input in enumerate(tqdm(val_loader)):



        if USE_CUDA:
            image = input['image'].cuda(non_blocking=True)
            target = input['target'].cuda(non_blocking=True)
            # location_data = input['location_data'].cuda(non_blocking=True)
            # map_square = input['map_square'].cuda(non_blocking=True)
        else:
            image = input['image']
            target = input['target']
            # location_data = input['location_data']
            # map_square = input['map_square']
        input_var = torch.autograd.Variable(image)
        target_var = torch.autograd.Variable(target)
        # location_data_var = torch.autograd.Variable(location_data)
        # map_square_var = torch.autograd.Variable(map_square)
        # compute output
        # output = model(input_var,map_square)
        output = model(input_var)

        output_aug = torch.nn.functional.softmax(output, dim=1)

        original_prediction = accuracy(output.data, target, topk=(1, 5,10))

        top1.update(original_prediction[0].item(), image.size(0))
        top5.update(original_prediction[1].item(), image.size(0))
        top10.update(original_prediction[2].item(), image.size(0))
        original_prediction = [int(op) for op in original_prediction[:-1]]
        original_prediction
        # for mul in [0.1,0.2,0.3,0.4,0.5]:

        for mul in [100]:
            output_aug = torch.nn.functional.softmax(output, dim=1)
            if mul not in better_1_4[square_res]:
                better_1_4[square_res][mul] = [AverageMeter(),AverageMeter(),AverageMeter()]

            id_ = int(input['Id'])
            
            # freq_square = validation_dat[validation_dat['Id']==int(input['Id'])][square_res].iloc[0]
            # freq_square = validation_dat[validation_dat['Id']==int(input['Id'])][square_res].iloc[0]
            # freq_square = test_dat[test_dat['Id']==int(input['Id'])][square_res].iloc[0]
            freq_square = test_dat[test_dat['Id']==int(input['Id'])][square_res].iloc[0]

            
            list_surrounding_squares = list(get_sourounding_squares(freq_square,square_res,test_dat)[square_res].unique())
            # try:
                
            freq_square_l = list(get_sourounding_squares(freq_square,square_res,test_dat)[square_res].unique())                
            freq_square_l_new = [f_s_l for f_s_l in freq_square_l if f_s_l in res_dat.columns]

            np_a = np.zeros(100)
            for c in res_dat[freq_square_l_new].columns:
                np_a += np.array(res_dat[freq_square_l_new][c])
            # np_freq_square = np.array(res_dat[freq_square])
# 
            output_aug[0] = output_aug[0]*100*torch.from_numpy(np_a).cuda()
            output_aug = nn.functional.normalize(output_aug)
            
            # except:
            #     print('e')
            #     pass
            # output_aug[0] = output_aug[0]+ (0.45)*torch.from_numpy(np.array(ast.literal_eval(validation_dat[validation_dat['path']==input['path'][0]]['in_1000'].values[0]))).cuda()
            # output_aug[0] = output_aug[0]+ (0.45/2)*torch.from_numpy(np.array(ast.literal_eval(test_dat[test_dat['path']==input['path'][0]]['in_1000'].values[0]))).cuda()
            # output_aug = nn.functional.normalize(output_aug)
            
            ##top-40 logistic
            # mul_log = multinomial_logisticmodel.predict_proba(df[df['Id']==int(id_)].loc[:, df.columns != 'Id'])[0]
            
            # ones = np.zeros(100)
            # null_val = np.argsort(mul_log)[60:]
            # # null_val = [i for i in null_val if i not in above_10_list]
            # ones[null_val]=1
            # mul_log[null_val]=0
            # output_aug[0] = output_aug[0]+ (0.8/2)*torch.from_numpy(ones).cuda()
            # output_aug = nn.functional.normalize(output_aug)




            augmented_prediction = accuracy(output_aug.data, target, topk=(1, 5, 10))
            top1_aug.update(augmented_prediction[0].item(), image.size(0))
            top5_aug.update(augmented_prediction[1].item(), image.size(0))
            top10_aug.update(augmented_prediction[2].item(), image.size(0))
            better_1_4[square_res][mul][0].update(augmented_prediction[0].item(), image.size(0))
            better_1_4[square_res][mul][1].update(augmented_prediction[1].item(), image.size(0))
            better_1_4[square_res][mul][2].update(augmented_prediction[2].item(), image.size(0))
#     
    for k in better_1_4[square_res]:
        print(square_res, k,better_1_4[square_res][k][0].avg)
warnings.filterwarnings("default")

In [ ]:
print(top1.avg)
print(top5.avg)
print(top10.avg)
# print(better_1)
for s in better_1_4:
    for k in better_1_4[s]:
        print(s,' &'+' 1','& ',round(better_1_4[s][k][0].avg,3),' & ',round(round(better_1_4[s][k][0].avg,3)-round(top1.avg,3),3),"\\\\")
        print('&'+' 5','& ',round(better_1_4[s][k][1].avg,3),' & ',round(round(better_1_4[s][k][1].avg,3)-round(top5.avg,3),3),"\\\\")
        print('&'+' 10','& ',round(better_1_4[s][k][2].avg,3),' & ',round(round(better_1_4[s][k][2].avg,3)-round(top10.avg,3),3),"\\\\")
        print("\\midrule")

In [ ]:
correct_dict ={}

In [ ]:
## testing different models with geo
##
##'


np.random.seed(0)


validation_or_test = 'validation'

if validation_or_test== 'test':
# validation_path = 'C:/Users/vjosv/master/dataset/images/csv/combined_lokasjon_top_100_images20_see1207_validation.csv'

    multinomial_logisticmodel = pickle.load(open('multinomial_logisticmodel1.sav', 'rb'))
    # multinomial_logisticmodel = pickle.load(open('multinomial_logisticmodel1_validate.sav', 'rb'))

    image_dataset = ImagesWithLocationDataset(test_dat_path,'s',transform=composed)


    val_loader = torch.utils.data.DataLoader(image_dataset, batch_size=1, shuffle=False, num_workers=0)
    data_frame = test_dat
    
if validation_or_test=='validation':

    data_frame = validation_dat
    multinomial_logisticmodel = pickle.load(open('multinomial_logisticmodel1_validate.sav', 'rb'))

    image_dataset = ImagesWithLocationDataset(validation_path,'s',transform=composed)


    val_loader = torch.utils.data.DataLoader(image_dataset, batch_size=1, shuffle=False, num_workers=0)

better_1_4 = {}
better_5 = {}
better_10 = {}

print(model.name)

def get_sourounding_squares(square,resolution, dat):
    if isinstance(square, str):
        square= literal_eval(square)
    
    return_list = []
    for i in [-1,0,1]:
        for j in [-1,0,1]:
            return_list.append(str([square[0]+i,square[1]+j]))
    
    return dat[dat[resolution].isin(return_list)]

torch.cuda.empty_cache()
model.eval()

import warnings
warnings.filterwarnings("ignore")

top1_aug = AverageMeter()
top5_aug = AverageMeter()
top10_aug = AverageMeter()

sq = ['square_20x20', 'square_15x15','square_10x10', 'square_30x30', 'square_40x40', 'square_50x50','square_60x60', 'square_70x70', 'square_80x80', 'square_90x90','square_100x100']
sq = ['square_150x150','square_200x200']


resnet_top1_list = []
resnet_top1_aug_list = []
resnet_top1_id_list = [] 

sq = ['squares_8','squares_20','squares_15', 'squares_10', 'squares_30', 'squares_40', 'squares_50','squares_60', 'squares_70', 'squares_80', 'squares_90', 'squares_100']
in_val = 'in_500'
sq = ['squares_100']

for square_res in sq:
    if square_res not in better_1_4:
        better_1_4[square_res] = {}
    res_dat = pd.read_csv(f'C:/Users/vjosv/master/Deep-Leafsnap/{square_res}_relative_freq_min.csv')
    top1 = AverageMeter()
    top5 = AverageMeter()
    top10 = AverageMeter()

    for i,input in enumerate(tqdm(val_loader)):



        if USE_CUDA:
            image = input['image'].cuda(non_blocking=True)
            target = input['target'].cuda(non_blocking=True)

        else:
            image = input['image']
            target = input['target']
            
        input_var = torch.autograd.Variable(image)
        target_var = torch.autograd.Variable(target)

        output = model(input_var)

        output_aug = torch.nn.functional.softmax(output, dim=1)

        original_prediction = accuracy(output.data, target, topk=(1, 5,10))

        top1.update(original_prediction[0].item(), image.size(0))
        top5.update(original_prediction[1].item(), image.size(0))
        top10.update(original_prediction[2].item(), image.size(0))
        original_prediction = [int(op) for op in original_prediction[:-1]]
        original_prediction
        # for mul in [0.1,0.2,0.3,0.4,0.5]:

        for mul in [5]:#[1,2,3,4,5,6,7,8,10,11,12,13]:
            output_aug = torch.nn.functional.softmax(output, dim=1)
            if mul not in better_1_4[square_res]:
                better_1_4[square_res][mul] = [AverageMeter(),AverageMeter(),AverageMeter()]

            id_ = int(input['Id'])
            

               
            if mul in [1,2,3,4,5,6,7,8,9,12]:
        
               
   
                freq_square = data_frame[data_frame['Id']==int(input['Id'])][square_res].iloc[0]
    
                freq_square_l = list(get_sourounding_squares(freq_square,square_res,data_frame)[square_res].unique())                
                freq_square_l_new = [f_s_l for f_s_l in freq_square_l if f_s_l in res_dat.columns]

                np_a = np.zeros(100)
                for c in res_dat[freq_square_l_new].columns:
                    np_a += np.array(res_dat[freq_square_l_new][c])
                # np_freq_square = np.array(res_dat[freq_square])
    # 
                output_aug[0] = output_aug[0]*100*torch.from_numpy(np_a).cuda()
                output_aug = nn.functional.normalize(output_aug)
            if mul in[1,2,3,4,5,9,10]:
                a= torch.from_numpy(np.array(ast.literal_eval(data_frame[data_frame['path']==input['path'][0]][in_val].values[0]))).cuda()
                for i in range(len(a)):
                    if a[i]==0:
                        a[i]=0.1
                output_aug[0] = output_aug[0]*a
                output_aug = nn.functional.normalize(output_aug)
            if mul in[7,11]:
                a= torch.from_numpy(np.array(ast.literal_eval(data_frame[data_frame['path']==input['path'][0]]['in_1500'].values[0]))).cuda()
                for i in range(len(a)):
                    if a[i]==0:
                        a[i]=0.1
                output_aug[0] = output_aug[0]*a
                output_aug = nn.functional.normalize(output_aug)
            # except:
            #     print('e')
            #     pass
            # if mul ==2:
            #     output_aug[0] = output_aug[0]+ (0.4)*torch.from_numpy(np.array(ast.literal_eval(test_dat[test_dat['path']==input['path'][0]]['in_1000'].values[0]))).cuda()
            # output_aug[0] = output_aug[0]+ (0.45/2)*torch.from_numpy(np.array(ast.literal_eval(test_dat[test_dat['path']==input['path'][0]]['in_1000'].values[0]))).cuda()
            # output_aug = nn.functional.normalize(output_aug)
            if mul ==2:
                output_aug[0] = output_aug[0]+ (0.6)*torch.from_numpy(np.array(ast.literal_eval(data_frame[data_frame['path']==input['path'][0]]['in_1000'].values[0]))).cuda()
            if mul in [3,6]:
                output_aug[0] = output_aug[0]+ (0.6/2)*torch.from_numpy(np.array(ast.literal_eval(data_frame[data_frame['path']==input['path'][0]]['in_1000'].values[0]))).cuda()
            
            # output_a
            ##top-40 logistic
            if mul in [2,5,7,8,9,10,11]:
                # try:
                mul_log = multinomial_logisticmodel.predict_proba(df[df['Id']==int(id_)].loc[:, df.columns != 'Id'])[0]

                ones = np.zeros(100)
                null_val = np.argsort(mul_log)[60:]
                # null_val = [i for i in null_val if i not in above_10_list]
                ones[null_val]=1
                output_aug[0] = output_aug[0]+(0.6)*torch.from_numpy(ones).cuda()
                #     print('s')
                # except:
                #     print('f')
                #     pass
            
            if mul in [3,4]:

                mul_log = multinomial_logisticmodel.predict_proba(df[df['Id']==int(id_)].loc[:, df.columns != 'Id'])[0]

                ones = np.zeros(100)
                null_val = np.argsort(mul_log)[60:]
                # null_val = [i for i in null_val if i not in above_10_list]
                ones[null_val]=1
            # mul_log[null_val]=0
                output_aug[0] = output_aug[0]+ (0.6/2)*torch.from_numpy(ones).cuda()

            # output_aug = nn.functional.normalize(output_aug)

            augmented_prediction = accuracy(output_aug.data, target, topk=(1, 5, 10))
        
            resnet_top1_list.append(original_prediction[0])
            resnet_top1_aug_list.append(int(augmented_prediction[0].item()))
            resnet_top1_id_list.append(id_)
            
            top1_aug.update(augmented_prediction[0].item(), image.size(0))
            top5_aug.update(augmented_prediction[1].item(), image.size(0))
            top10_aug.update(augmented_prediction[2].item(), image.size(0))
            better_1_4[square_res][mul][0].update(augmented_prediction[0].item(), image.size(0))
            better_1_4[square_res][mul][1].update(augmented_prediction[1].item(), image.size(0))
            better_1_4[square_res][mul][2].update(augmented_prediction[2].item(), image.size(0))
#     
    for k in better_1_4[square_res]:
        print(square_res, k,better_1_4[square_res][k][0].avg)
        
correct_dict['resnet_org']=resnet_top1_list
correct_dict['resnet_aug']=resnet_top1_aug_list
correct_dict['resnet_ids'] = resnet_top1_id_list
warnings.filterwarnings("default")

In [ ]:
print(top1.avg)
print(top5.avg)
print(top10.avg)
# print(better_1)
for s in better_1_4:
    for k in better_1_4[s]:
        print(k)
        print(' 1','& ',round(better_1_4[s][k][0].avg,3),'& ',round(round(better_1_4[s][k][0].avg,3)-round(top1.avg,3),3),"\\\\")
        print(' 5','& ',round(better_1_4[s][k][1].avg,3),'& ',round(round(better_1_4[s][k][1].avg,3)-round(top5.avg,3),3),"\\\\")
        print(' 10','& ',round(better_1_4[s][k][2].avg,3),'& ',round(round(better_1_4[s][k][2].avg,3)-round(top10.avg,3),3),"\\\\")
        print("\\midrule")